## This notebook is part of self learning Quantum Computing Algorithms

## Description: This program optimizes the stock portfolio
### With real time data + increase in assests

In [10]:
import numpy as np
from qiskit_finance.data_providers import RandomDataProvider
from pandas_datareader import data
import datetime
import pandas as pd

In [42]:
num_assets = 6
seed = 123
start=datetime.datetime(2021,6,1)
end=datetime.datetime(2021,6,30)
# Generate expected return and covariance matrix from (random) time-series
stocks = ['AAPL', 'NKE', 'GOOGL', 'AMZN', 'FB', 'MSFT']

In [43]:
# Import data
df = data.DataReader(stocks, 'yahoo', start=start, end=end)

In [44]:
df.head()

Attributes   Adj Close                                                    \
Symbols           AAPL         NKE        GOOGL         AMZN          FB   
Date                                                                       
2021-06-01  124.094070  134.509995  2381.179932  3218.649902  329.130005   
2021-06-02  124.872902  134.169998  2370.590088  3233.989990  329.149994   
2021-06-03  123.355186  134.169998  2347.580078  3187.010010  326.040009   
2021-06-04  125.701668  133.740005  2393.570068  3206.219971  330.350006   
2021-06-07  125.711655  133.949997  2402.300049  3198.010010  336.579987   

Attributes                   Close                                        ...  \
Symbols           MSFT        AAPL         NKE        GOOGL         AMZN  ...   
Date                                                                      ...   
2021-06-01  246.927277  124.279999  134.509995  2381.179932  3218.649902  ...   
2021-06-02  246.827469  125.059998  134.169998  2370.590088  3233.989990  ...   
2021-06-03  245.240524  123.540001  134.169998  2347.580078  3187.010010  ...   
2021-06-04  250.310791  125.889999  133.740005  2393.570068  3206.219971  ...   
2021-06-07  253.325027  125.900002  133.949997  2402.300049  3198.010010  ...   

Attributes         Open                                         Volume  \
Symbols           GOOGL         AMZN          FB        MSFT      AAPL   
Date                                                                     
2021-06-01  2374.439941  3243.500000  330.149994  251.229996  67637100   
2021-06-02  2389.149902  3223.100098  330.380005  248.130005  59278900   
2021-06-03  2345.729980  3204.229980  325.779999  245.220001  76229200   
2021-06-04  2369.270020  3212.000000  325.899994  247.759995  75169300   
2021-06-07  2389.439941  3197.330078  329.480011  249.979996  71057600   

Attributes                                                 
Symbols         NKE    GOOGL     AMZN        FB      MSFT  
Date                                                       
2021-06-01  5577900  1167700  2430000  11765900  23213300  
2021-06-02  5226100  1057900  2014500  11654300  19406700  
2021-06-03  5027400   934800  2398300  12610800  25307700  
2021-06-04  5217100  1222900  2249700  13289400  25281100  
2021-06-07  3765000  1206000  2215800  20136700  23079200  

[5 rows x 36 columns]

In [45]:
df = df['Adj Close']

In [46]:
df.head()

Symbols,AAPL,NKE,GOOGL,AMZN,FB,MSFT
Date,,,,,,
2021-06-01,124.094070,134.509995,2381.179932,3218.649902,329.130005,246.927277
2021-06-02,124.872902,134.169998,2370.590088,3233.989990,329.149994,246.827469
2021-06-03,123.355186,134.169998,2347.580078,3187.010010,326.040009,245.240524
2021-06-04,125.701668,133.740005,2393.570068,3206.219971,330.350006,250.310791
2021-06-07,125.711655,133.949997,2402.300049,3198.010010,336.579987,253.325027


In [47]:
df_data = []
for stock in stocks:
    df_data.append(df[stock].values.tolist())
print(df_data)

[[124.09407043457031, 124.8729019165039, 123.35518646240234, 125.70166778564453, 125.71165466308594, 126.5503921508789, 126.93981170654297, 125.92134094238281, 127.15947723388672, 130.2847900390625, 129.44606018066406, 129.95529174804688, 131.59283447265625, 130.26483154296875, 132.10208129882812, 133.77955627441406, 133.49998474121094, 133.21041870117188, 132.91087341308594, 134.578369140625, 136.1260528564453, 136.75511169433594], [134.50999450683594, 134.1699981689453, 134.1699981689453, 133.74000549316406, 133.9499969482422, 133.35000610351562, 131.83999633789062, 130.97999572753906, 131.94000244140625, 131.36000061035156, 130.2899932861328, 130.39999389648438, 128.9199981689453, 128.41000366210938, 130.0800018310547, 132.47999572753906, 133.10000610351562, 133.60000610351562, 154.35000610351562, 152.36000061035156, 155.9499969482422, 154.49000549316406], [2381.179931640625, 2370.590087890625, 2347.580078125, 2393.570068359375, 2402.300048828125, 2398.43994140625, 2407.93994140625,

## $\min\limits_{x \in \{0, 1\}^{n}} q \ x^{T}\Sigma x − \mu^{T}x \\ \text{or} \\ \max\limits_{x \in \{0, 1\}^{n}} \mu^{T}x - q \ x^{T}\Sigma x \\ \text{subject to}: 1^{T} x = B$

##### $\text{ref: https://qiskit.org/documentation/tutorials/finance/01_portfolio_optimization.html} \\  x \in \{0, 1\}^{n} \ \text{denotes the vector of binary decision variables, which indicate which assets to pick} \left(x[i]=1\right) \text{and which not to pick} \left(x[i]=0\right), \\ \mu \in R^{n} \ \text{defines the expected returns for the assets}, \\ \Sigma \in R^{n \times n} \text{specifies the covariances between the assets}, \\ q \gt 0 \ \text{controls the risk appetite of the decision maker}, \\ \text{and B denotes the budget, i.e. the number of assets to be selected out of n.}$

## $\text{The equality constraint} \ 1^{T}x = B \ \text{is mapped to a penalty term} \ \left(1^{T} x − B\right)^{2}$

## $\text{Assumptions}:\\ \ \cdot \text{all assets have the same price} \left(\text{normalized to 1}\right) \\ \ \cdot \text{the full budget B has to be spent, i.e., one has to select exactly B assets.}$

In [22]:
def divide_2(val_1, val_2):
    if val_2 == 0:
        if val_1 == 0:
            return 1
        return np.nan
    return val_1 / val_2

## Calculate mean returns 
## $\mu \in R^{n} \ \text{defines the expected returns for the assets}$

In [5]:
div_func = np.vectorize(divide_2)
period_returns = div_func(np.array(data)[:, 1:], np.array(data)[:, :-1]) - 1
mu = np.mean(period_returns, axis=1)

## Calculate mean covariance
##  $\Sigma \in R^{n \times n} \text{specifies the covariances between the assets}$

In [49]:
div_func = np.vectorize(divide_2)
period_returns = div_func(np.array(df_data)[:, 1:], np.array(df_data)[:, :-1]) - 1
sigma = np.cov(period_returns)

## plot sigma

In [27]:
import matplotlib.pyplot as plt

In [48]:
div_func = np.vectorize(divide_2)
period_returns = div_func(np.array(df_data)[:, 1:], np.array(df_data)[:, :-1]) - 1
mu = np.mean(period_returns, axis=1)

## $q \gt 0 \ \text{controls the risk appetite of the decision maker}$

In [24]:
q = 0.5 # set risk factor

## $\text{B denotes the budget, i.e. the number of assets to be selected out of n.}$

In [26]:
budget = 2 # set budget (B)

### ref: https://qiskit.org/documentation/tutorials/finance/01_portfolio_optimization.html

In [50]:
penalty = num_assets # set parameter to scale the budget penalty term

## Model

In [29]:
from docplex.mp.model import Model

In [51]:
mdl = Model('portfolio model')
#x = mdl.binary_var_list(num_assets)

In [52]:
x = list()
for i in range(num_assets):
    x.append(mdl.binary_var(name="x_{0}".format(i)))
print(x)

[docplex.mp.Var(type=B,name='x_0'), docplex.mp.Var(type=B,name='x_1'), docplex.mp.Var(type=B,name='x_2'), docplex.mp.Var(type=B,name='x_3'), docplex.mp.Var(type=B,name='x_4'), docplex.mp.Var(type=B,name='x_5')]


## $\max\limits_{x \in \{0, 1\}}  \mu^{T} * x - q * x^{T} * \Sigma * x \\ \text{linear} = c^{T}x, \text{qudratic} = x^{T}Qx = \sum\limits_{i, j = 1}^{n} x_{i}^{T} \Sigma_{ij} x_{j} \\ \text{objective} = \text{linear - }\left(\text{risk_factor * qudratic}\right)$

In [53]:
linear = np.dot(mu, x) # mu^T * x
qudratic = mdl.sum([x[i]*sigma[i][j]*x[j] for i in range(num_assets) for j in range(num_assets)])
objective = linear - q * qudratic
mdl.maximize(objective)

## budget constraint:  $\sum\limits_{i =1}^{n} 1^{T}x_{i} == \text{budget}$

In [54]:
cost = mdl.sum([x[i] for i in range(num_assets)])
mdl.add_constraint(cost == budget, ctname='budget')

docplex.mp.LinearConstraint[budget](x_0+x_1+x_2+x_3+x_4+x_5,EQ,2)

## converting to Quadratic Program

## removing the constraint to create the QUBO

In [35]:
from qiskit_optimization.translators import from_docplex_mp

In [55]:
mod = from_docplex_mp(mdl)

In [56]:
print(mod.export_as_lp_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: portfolio model

Maximize
 obj: 0.004682261762 x_0 + 0.007159640330 x_1 + 0.001227757962 x_2
      + 0.003234622966 x_3 + 0.002725277591 x_4 + 0.004364434718 x_5 + [
      - 0.000095526386 x_0^2 + 0.000075990183 x_0*x_1 - 0.000091374658 x_0*x_2
      - 0.000085351262 x_0*x_3 - 0.000138108962 x_0*x_4 - 0.000097158652 x_0*x_5
      - 0.001244762113 x_1^2 + 0.000012725679 x_1*x_2 + 0.000326101317 x_1*x_3
      + 0.000149750656 x_1*x_4 + 0.000144573046 x_1*x_5 - 0.000063470332 x_2^2
      - 0.000047494852 x_2*x_3 - 0.000129234354 x_2*x_4 - 0.000111986840 x_2*x_5
      - 0.000126959488 x_3^2 - 0.000100278531 x_3*x_4 - 0.000069931804 x_3*x_5
      - 0.000226327311 x_4^2 - 0.000182207269 x_4*x_5 - 0.000071725030 x_5^2 ]/2
Subject To
 budget: x_0 + x_1 + x_2 + x_3 + x_4 + x_5 = 2

Bounds
 0 <= x_0 <= 1
 0 <= x_1 <= 1
 0 <= x_2 <= 1
 0 <= x_3 <= 1
 0 <= x_4 <= 1
 0 <= x_5 <= 1

Binaries
 x_0 x_1 x_2 x_3 x_4 x_5
End



## QAOA

In [38]:
from qiskit.utils import algorithm_globals, QuantumInstance
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit_optimization.converters import QuadraticProgramToQubo
from qiskit.algorithms import QAOA
from qiskit.algorithms.optimizers import ADAM, COBYLA
from qiskit import Aer

In [57]:
def index_to_selection(i, num_assets):
    s = "{0:b}".format(i).rjust(num_assets)
    x = np.array([1 if s[i]=='1' else 0 for i in reversed(range(num_assets))])
    return x

In [58]:
def print_result(result):
    selection = result.x
    res = list()
    for i in range(len(selection)):
        if selection[i]:
            res.append(stocks[i])
    value = result.fval
    print('Optimal: selection {}, value {:.4f}'.format(selection, value))
    print('Optimal: selected stocks {}'.format(res))

    eigenstate = result.min_eigen_solver_result.eigenstate
    eigenvector = eigenstate if isinstance(eigenstate, np.ndarray) else eigenstate.to_matrix()
    probabilities = np.abs(eigenvector)**2
    i_sorted = reversed(np.argsort(probabilities))
    print('\n----------------- Full result ---------------------')
    print('selection\tvalue\t\tprobability')
    print('---------------------------------------------------')
    for i in i_sorted:
        x = index_to_selection(i, num_assets)
        value = QuadraticProgramToQubo().convert(mod).objective.evaluate(x)
        probability = probabilities[i]
        print('%10s\t%.4f\t\t%.4f' %(x, value, probability))

In [59]:
algorithm_globals.random_seed = 1234
backend = Aer.get_backend('statevector_simulator')
cobyla = COBYLA()
cobyla.set_options(maxiter=250)
quantum_instance = QuantumInstance(backend=backend, seed_simulator=seed, seed_transpiler=seed)
qaoa_mes = QAOA(optimizer=cobyla, reps=3, quantum_instance=quantum_instance)
eigen_optimizer = MinimumEigenOptimizer(min_eigen_solver = qaoa_mes)
result = eigen_optimizer.solve(mod)
print_result(result)

Optimal: selection [1. 1. 0. 0. 0. 0.], value 0.0112
Optimal: selected stocks ['AAPL', 'NKE']

----------------- Full result ---------------------
selection	value		probability
---------------------------------------------------
[1 0 0 0 0 1]	-0.0089		0.0564
[0 1 0 1 0 0]	-0.0099		0.0564
[1 0 0 0 1 0]	-0.0072		0.0564
[1 0 1 0 0 0]	-0.0058		0.0564
[0 1 0 0 0 1]	-0.0109		0.0564
[0 1 0 0 1 0]	-0.0092		0.0564
[0 0 1 0 0 1]	-0.0055		0.0564
[1 1 0 0 0 0]	-0.0112		0.0564
[0 0 0 0 1 1]	-0.0068		0.0564
[0 0 1 0 1 0]	-0.0037		0.0564
[0 0 0 1 0 1]	-0.0075		0.0564
[1 0 0 1 0 0]	-0.0078		0.0564
[0 0 1 1 0 0]	-0.0043		0.0564
[0 0 0 1 1 0]	-0.0057		0.0564
[0 1 1 0 0 0]	-0.0077		0.0563
[1 0 0 0 0 0]	1.0206		0.0127
[0 0 0 0 0 1]	1.0209		0.0127
[0 0 0 0 1 0]	1.0226		0.0127
[0 1 0 0 0 0]	1.0187		0.0127
[0 0 0 1 0 0]	1.0220		0.0127
[0 0 1 0 0 0]	1.0240		0.0127
[0 0 0 0 0 0]	4.1008		0.0111
[1 1 0 0 0 1]	1.0096		0.0024
[0 1 0 1 0 1]	1.0110		0.0024
[1 1 0 1 0 0]	1.0107		0.0024
[1 1 0 0 1 0]	1.0114		0.0024
[0 

## If you have any queries on this notebook please reach to me bala.na@hcl.com